# 编码分类特征

在机器学习中,特征经常不是数值型的而是定类型(nominal)的.比如,一个人可能有 ["male", "female"],["from Europe", "from US", "from Asia"]等定类型的特征.这些特征能够被有效地编码成整数,比如["male", "from US"]可以被表示为[0, 1],["female", "from Asia"]表示为[1, 2,].

这个整数特征表示并不能直接作为特征在机器学习中使用,因为这样的连续输入,模型会认为类别之间是定距型(scala),但实际上既是无序的更是无距的.

假设一个定类型数据有K种可能取值,将其转换为能够被机器学习模型使用的编码有
- 独热编码(one-hot encoding), 在OneHotEncoder中实现.每个类别被转化为一个K维向量,每维对应一种取值.每种类别转化为该类对应的维度为1,其他维度为0的K维向量.
- 虚拟编码(dummy coding), 选出一种可能取值作为参照取值(叫做参照类).每个类别被转化为一个K-1维向量,每个维度对应除了参照类之外的一种取值.每种非参照类类别转化为该类对应的维度为1,其他维度为0的K-1维向量.参照类转化为K-1维零向量.
- 效应编码(effect coding),和虚拟编码一样选出一个参照类,除了参照类的取值编码方式一样,只是参照类转化为K-1维全是-1的向量.




# 使用不同分类变量编码时类别作为特征时的线性回归

| 编码方式| 截距 |  i类系数 | i类编码,共K类| 编码维度 |
|------ | -------| -------| -------- |   --------| 
| dummy code |  参照类均值 | i类的均值 - 参照类均值 | 若不是参照类,i-dim为1,其他为0;否则全为 0 | K-1 |
| effect | 总体均值 | i类均值 - 总体均值 | 若不是参照类,i-dim为1,其他为0;否则全部为-1 |   K-1|
| one-hot | 总体均值 + c | i类均值-总体均值 - c| i-dim为1,其他为0 | K |

总体均值指的是所有类均值的均值, c为任意常数

sklearn中相关的接口有:

接口|说明
---|---
`preprocessing.OneHotEncoder([n_values, …])`|独热编码,要求训练和转换的参数为int型,输出的为
`preprocessing.LabelBinarizer([neg_label, …])`|将枚举型特征转换为二值化特征矩阵,输入为int型或者字符串型
`preprocessing.MultiLabelBinarizer([classes, …])`|将复数枚举型特征转换为二值化特征矩阵,输入为int型或者字符串型组成的set
`preprocessing.LabelEncoder`|将枚举型特征编码,输入为int型或者字符串型

pandas中dummy coding的接口:

接口|说明
---|---
`pandas.get_dummies` | 虚拟编码

## 虚拟编码

In [4]:
import pandas as pd

df = pd.DataFrame(
    {
        'A': ['a', 'b', 'c'], 
        'B': ['b', 'a', 'c'],
        'C': [1, 2, 3]
    }
)

pd.get_dummies(df, prefix=['col1', 'col2'], drop_first=False)

,C,col1_a,col1_b,col1_c,col2_a,col2_b,col2_c
0,1,1,0,0,0,1,0
1,2,0,1,0,1,0,0
2,3,0,0,1,0,0,1


## 独热编码

In [1]:
from sklearn import preprocessing

In [7]:
enc = preprocessing.OneHotEncoder()
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])  
enc.transform([[0, 1, 3]]).toarray()

array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])

## 枚举型特征编码

### 针对int类型

In [3]:
le = preprocessing.LabelEncoder()
le.fit([1, 2, 2, 6])
le.classes_

array([1, 2, 6])

In [5]:
le.transform([1, 1, 2, 6])# 编码

array([0, 0, 1, 2], dtype=int64)

In [6]:
le.inverse_transform([0, 0, 1, 2]) # 解码

array([1, 1, 2, 6])

### 针对str类型

In [8]:
le = preprocessing.LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
list(le.classes_)

['amsterdam', 'paris', 'tokyo']

In [9]:
le.transform(["tokyo", "tokyo", "paris"])

array([2, 2, 1], dtype=int64)

In [10]:
list(le.inverse_transform([2, 2, 1]))

['tokyo', 'tokyo', 'paris']

## 二值化特征

### 针对单一枚举类型

In [11]:
lb = preprocessing.LabelBinarizer()
lb.fit([1, 2, 6, 4, 2])
lb.classes_

array([1, 2, 4, 6])

In [12]:
lb.transform([1, 6])

array([[1, 0, 0, 0],
       [0, 0, 0, 1]])

### 针对多枚举类型

In [13]:
lb = preprocessing.MultiLabelBinarizer()
lb.fit_transform([(1, 2), (3,)])
lb.classes_

array([1, 2, 3])